# Access control
There are several layers in a computer system:
* Applications (*eg* Google Chrome, Microsoft Word)
* Services (*eg* Java Virtual Machine, Pulse Audio)
* Operating Systems (*eg* Windows, macOS, UNIX/Linux)
* OS Kernel (*ie* system calls, such as memory management)
* Hardware (*eg* CPU, RAM, SSD, I/O)

## Comparison with network layers

|    | Communication network | Computer System |
|----| ---- | ---- |
| Well defined boundary? | More well defined | Less well defined |
| Data flow | Data flows from top to the bottom layer | Each layer has its own process and data |
| Concern | Data confidentiality and integrity | Data confidentiality and integrity, and **process integrity** | 

An example of the layers being less defined in computer system is between the OS and the kernel.
Some literature consider them part of the same layer.

**Process integrity** is the assurance that the process will not deviate from its intended execution path.

The layers are arranged from the least privileged (application) to the most privileged (hardware).
Thus, a secure system would be such that if one of the layers is compromised by an attacker, they are not able to manipulate the objects in the inner layers.
(Note that this is rather difficult to achieve, due to numerous issues such as implementation errors, user error *etc*)

One can imagine the chaos if an attacker who is able to perform SQL injection on the DBMS (service layer) is somehow able to obtain the password file (OS layer) of the system through it.
Or if an attacker that employed cross-site scripting to compromised the browser (application layer) is able to burn out the CPU (hardware layer).

## Access control model
Access control are required in a computer system to restrict the **operations** that can be performed by some **entity** on some **objects**.

Operations can be categorized as:
* Observe (*eg* reading a file)
* Alter (*eg* writing to a file, replacing a file, deleting a file, changing ownership)
* Action (*eg* executing a file)

Suppose that a **subject/principal** wants perform some **operation** on some **object**.
An example would be the user with name `John` wishes to `read` the file `johns_grades.txt`. 
The entity known as the **reference monitor** is responsible to deciding whether to allow or deny the access.

There are two main types of access control

### Discretionary access control model
The owner of the object decides the rights

### Mandatory access control model
A system-wide policy that decides the rights.

## Access control representation
An **access control matrix** shows the relationship between principals and objects.

|  | sudo | passwd | common.txt |
| --- | ---| --- | --- |
|root | {run} | {read, write} | {read, write} |
| Alice | {run} | {read} | {read, write} |
| Bob | {} |  {read} | {read, write} |

This matrix can be represented in two ways.

### Access control list
Stores the rights to a particular object as a list.

```
sudo: [(root, {run}), (Alice, {run})]
passwd: [(root, {read, write}), (Alice, {read}, (Bob, {read})]
common.txt: [(root, {read, write}), (Alice, {read, write}), (Bob, {read, write})]
```

### Capabilities
Stores the capabilities of each subject as a list.

```
root: [(sudo, {run}), (passwd, {read, write}), (common.txt, {read, write})]
Alice: [(sudo, {run}), (passwd, {read}), (common.txt, {read, write})]
Bob: [(passwd, {read}), (common.txt{read, write})]
```

Note that is is difficult to know which objects does a particular subject has access to in ACL, but easy in capabilities.
The vice versa occurs when finding who has access to a certain object.

However, notice that the matrix can get really large if there are multiple users, and there are numerous files.

Thus, to help simplify the matrix representation, we can group the users and define access rights to these groups instead.

## UNIX access control
File permissions consist of rights to the following user classes:
* owner
* owner's group(s)
* others

In UNIX, groups can only be created by `root`.

Because all resources are treated as files in UNIX, we can define access controls on these resources (hardware, I/O) the same way we do for files.

Each user:
* has a unique username
* has a unique user identifier (**UID**)
    * stored in `/etc/passwd`
* can belong to one or more groups
    * first group is stored in `/etc/passwd`
    * other groups are stored in`/etc/group`
    
Each group:
* has a unique group name
* has a unique group identifier (**GID**)

Purpose of UID/GID:
* determine ownership of system resources
* determine credential of running processes
* control permissions granted to processes

There is a special user called the **super use**, with UID 0, typically called `root`.
All security checks are disabled for the super user.

### passwd file
The `passwd` file is made readable by everyone because some processes requires information in it.

In older versions of UNIX, the password hash is stored in the file.
This allowed attackers to perform offline password guessing to crack the password.

In newer versions, the hash is stored elsewhere, typically in `/etc/shadow`.
This hash is not readable by everyone.

### shadow file

Within the shadow file, each entry is formated as follows (separated by `:`)
* login name
* hashed password
* date of last password change, 
* minimum password age
* maximum password age
* password warning period
* password inactivity period
* account expiration date
* reserved field 

For the second field (hashed password), has the following format: 

`$id$salt$hash`

where `id` correspond to the hash-method used (5=SHA-256, 6= SHA-512, *etc*)
    

### Processes
A new process is spawned when running an executable file, or a child is forked from a parent process.

Each process has its own **process ID (PID)**.
Use `ps aux` to see all processes and their PID.

### File permissions
File permissions are represented by 9 characters, with each triplet corresponding to a certain user class.
The first triplet corresponds to the owner; the next to users in the group and the last to everyone else.

Each triplet corresponds to the three actions, `r` read, `w` write, `x` execute.

In [47]:
!exa -l

drwxr-xr-x    - own3d 10 Jul 11:19 access-control-example
.rw-r--r--  25k own3d 10 Jul 11:45 access_control.ipynb
.rw-r--r--  17k own3d  6 Jul 08:28 authentication.ipynb
.rw-r--r--  75k own3d 26 Jun 11:12 classical_ciphers.ipynb
.rw-r--r--  25k own3d  6 Jul 23:49 data_origin_authentication.ipynb
.rw-r--r-- 4.2k own3d  7 Jul 23:16 introduction.ipynb
.rw-r--r-- 1.4M own3d  6 Jul 09:40 modern_ciphers.ipynb
drwxr-xr-x    - own3d  6 Jul 09:27 module
.rw-r--r--  12k own3d  9 Jul 00:48 network_security.ipynb
.rw-r--r--  13k own3d  6 Jul 11:33 public_key_infrastructure.ipynb
.rw-r--r--  148 own3d 26 Jun 11:27 README.md
.rw-r--r--  11k own3d  7 Jul 00:18 secure_channel.ipynb


Note that I use `exa` instead of `ls` because it is more convenient for everyday use.

#### Special permissions
##### Set-UID <span id='set-uid'/>
Represented by an `s` replacing the owner's `x` bit, this causes the process' **effective UID** to be the owner's, rather than the user running it.

##### Set-GID
Represented by an `s` replacing the group's `x` bit, this causes the process' **effective GID** to be the group owner.

##### Sticky bit
Represented by an `t` replacing the other's `x` bit, if set on a directory, only the owner or root can delete the files in that directory.

Note that only the owner or root can change the permission of a file.

A process has its **process credentials**, determined by its **real UID** and **effective UID**.

Real UID is inherited from the user who ran the process, which identifies the real owner of the process.

When set-UID bit is not set, the effective UID of the process is the real UID.

When set-UID bit is set, the effective UID of the process is the file owner's UID.

#### Purpose of set-UID
Suppose we have a password file, which contains all the user's password.
Thus, it makes sense to not make it readable/writable to everyone.

However, suppose that we also are required to allow users to change their own passwords.
Because the file contains everyone else's passwords, we cannot allow it to be writable to the user, thus we cannot satisfy the above requirement.

Hence, `root` can create special program `change_password` which interacts with the password file.
This file is made executable by all the users.
To allow the password file to be modified when the user runs the program, we set the set-UID bit of the program.

With the set-UID bit, the process inherits the UID of `root`, allowing access to the password file in a controlled manner through this program.
This allows **temporary privilege escalation** of the user when they run the program.

Since the privilege of the user is temporarily escalated, it is important to ensure that there is no vulnerabilities in the program.
If the attacker finds a vulnerability, they can use it to perform malicious actions that they otherwise were not able to perform due to insufficient privileges.
These are known as **privilege escalation attacks**

##### Example

In [36]:
%cd access-control-example
from getpass import getpass

[Errno 2] No such file or directory: 'access-control-example'
/home/own3d/wellspring/cyber_security/access-control-example


This is our file structure.

In [37]:
!exa -l

.rwsr-xr-x 53k root  10 Jul 11:14 change_secret_name
.rw-r--r-- 359 own3d 10 Jul 11:14 change_secret_name.py
.rw-------  52 root  10 Jul 11:19 secret_names.txt


Within `secret_names.txt`, there is a list of users and their corresponding secret names.
We require that no one is able to peek at the other's secret name, thus the file is not world-readable and not world-writable.

(Note that all commands which executes using `sudo` is not supposed to be allowed by the regular user. We need it to display the contents of the file for demonstration)

In [38]:
!echo {getpass()} | sudo -S cat secret_names.txt | sed '1 i\ '

········
[sudo] password for own3d:  
name	secret name
own3d	0wn463
alice	diana
bob	clark


In [39]:
!cat secret_names.txt

cat: secret_names.txt: Permission denied


Thus, we employ a binary with a set-uid bit set (take note of the set-uid bit in the file permission) to allow controlled access to the `secret_names.txt` file

`change_secret_name` is the compile C binary using `change_secret_name.py` as the source code.
We need to compile the Python script into C because the Linux kernel does not inherit set-uid privileges for interpreted scripts, such as Bash and Python.

In [40]:
!cat change_secret_name.py

import os
import sys
import re

SECRET_FILE = 'secret_names.txt'

with open(SECRET_FILE, 'r') as f:
    data = f.read()

user = os.getenv('USER')
new_secret = sys.argv[1]

print(f'{user} is changing their secret name to {new_secret}')

new_data = re.sub(rf'{user}\t.*', rf'{user}\t{new_secret}', data)

with open(SECRET_FILE, 'w') as f:
    f.write(new_data)


###### Changing secret name


In [41]:
!./change_secret_name new_name

own3d is changing their secret name to new_name


In [42]:
!echo {getpass()} | sudo -S cat secret_names.txt | sed '1 i\ '

········
[sudo] password for own3d:  
name	secret name
own3d	new_name
alice	diana
bob	clark


As we can see, we can use the executable to change the name without having direct root access.

###### Vulnerability

However, some keen users may have already seen the vulnerability in the program.
It assumed that the user of the executable corresponds to the `USER` variable.
However, the `USER` variable can be modified by the caller of the executable, like so.

In [43]:
!USER=alice ./change_secret_name enemies_stand

alice is changing their secret name to enemies_stand


In [44]:
!echo {getpass()} | sudo -S cat secret_names.txt | sed '1 i\ '

········
[sudo] password for own3d:  
name	secret name
own3d	new_name
alice	enemies_stand
bob	clark


Thus, I am able to modify `alice`'s secret name even though I am not `alice`.